In [1]:
# Linear Regression, Ridge & Lasso
%pylab inline
import numpy as np
import pandas as pd
from sklearn import neighbors
from numpy import genfromtxt
import scipy.spatial.distance as dist
import random
from sklearn.cluster import KMeans
from PIL import Image
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# SVM, Logistic Regression
from sklearn import svm
from sklearn import model_selection
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler
import glob

Populating the interactive namespace from numpy and matplotlib


In [2]:
import csv

# get data
filepath = 'DummyData.csv'
data = []  
f = open(filepath) 
reader = csv.reader(f)
for row in reader:
    data.append(row)
f.close()  

# Get the variables list
varList = data[0]

# Create the list of output variables
'''
# output variables 
# w2b1401
# w2cogscore w2cog3pl
# w2chn w2upchn
# w2mat w2upmat
# w2eng w2upeng
'''
# outVarList = ['w2b1401', 'w2cogscore','w2cog3pl', 'w2chn', 'w2upchn', 'w2mat', 'w2upmat', 'w2eng', 'w2upeng']

# Used for test
UsedForTest = ['w2stdtotal']

# Load the data
# trainp = genfromtxt('trainp.csv', delimiter=',')
data = genfromtxt('Today.csv',  delimiter=',')

input_num = data.shape[0]
var_num = data.shape[1]

# Take 2/3 data as train set, 1/3 as test set
train_data_num = int(input_num * 2/3)

# Without cross-validation

# Training data: 
trainfeat=data[:train_data_num, :] #Training features (rows are feature vectors)
trainresp=data[:train_data_num, :] #Training responses

# Validation data:
valfeat=data[train_data_num + 1:, :] #Validation Features (rows are feature vectors)
valresp=data[train_data_num + 1:, :] #Validation Response

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(open('DummyData.csv'))
data = data.dropna()                                                                               
X = data.values[:,5:-1]
Y = data.values[:,4]
colnames = list(data.columns[5:-1])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

### Linear Regression

In [37]:
from sklearn import linear_model

lmodel = linear_model.LinearRegression()
lmodel.fit(X_train, Y_train)
lmodel.coef_
lmodel.intercept_
coef = [lmodel.intercept_]
coef.extend(list(lmodel.coef_))
cols = ["interccept"]
cols.extend(colnames)
coef = pd.DataFrame(coef, index = cols, columns = ["coefficient"])
coef.to_csv("linear_reg_coef.csv")

In [6]:
# Root of Mean Square Error 
train_err_l = np.sqrt(np.mean((lmodel.predict(X_train) - Y_train)**2))
test_err_l =np.sqrt(np.mean((lmodel.predict(X_test) - Y_test)**2))
train_var_pct_l = np.var(np.array(lmodel.predict(X_train))) / np.var(Y_train)
test_var_pct_l = np.var(np.array(lmodel.predict(X_test))) / np.var(Y_test)

In [7]:
train_var_pct_l, test_var_pct_l

(0.29379590015079643, 0.32132174386808326)

In [8]:
train_err_l, test_err_l

(0.6428677934077509, 0.6446420574518552)

### Neural Network

In [8]:
# Neural Network
from sklearn.neural_network import MLPRegressor as mlp

Train_err_n = []
Test_err_n = []
for i in range(10,100,5):
    nmodel=mlp(activation= 'logistic',learning_rate_init=0.005,
               random_state=66, max_iter=1000, hidden_layer_sizes=(i))
    nmodel.fit(X_train,Y_train)
    Train_err_n.append(np.sqrt(np.mean((nmodel.predict(X_train) - Y_train)**2)))
    Test_err_n.append(np.sqrt(np.mean((nmodel.predict(X_test) - Y_test)**2)))

import matplotlib.pyplot as plt

plt.plot(range(10,100,5),Train_err_n)
plt.title("Training Error of Neural Network")
plt.xlabel("Number of neurons")
plt.ylabel("Error")
plt.savefig("nn-train-err.png")
plt.close()

plt.plot(range(10,100,5),Test_err_n)
plt.title("Test Error of Neural Network")
plt.xlabel("Number of neurons")
plt.ylabel("Error")
plt.savefig("nn-test-err.png")
plt.close()

### Results
nmodel=mlp(activation= 'logistic',learning_rate_init=0.005,
           random_state=66, max_iter=1000, hidden_layer_sizes=(40))
nmodel.fit(X_train,Y_train)

train_err_n = np.sqrt(np.mean((nmodel.predict(X_train) - Y_train)**2))
test_err_n = np.sqrt(np.mean((nmodel.predict(X_test) - Y_test)**2))
train_var_pct_n = np.var(np.array(nmodel.predict(X_train))) / np.var(Y_train)
test_var_pct_n = np.var(np.array(nmodel.predict(X_test))) / np.var(Y_test)

In [7]:
from sklearn.neural_network import MLPRegressor as mlp

nnmodel=mlp(activation= 'logistic',learning_rate_init=0.005,
           random_state=66, max_iter=1000, hidden_layer_sizes=(90))
nnmodel.fit(X_train,Y_train)

train_err_n = np.sqrt(np.mean((nnmodel.predict(X_train) - Y_train)**2))
test_err_n = np.sqrt(np.mean((nnmodel.predict(X_test) - Y_test)**2))
train_var_pct_n = np.var(np.array(nnmodel.predict(X_train))) / np.var(Y_train)
test_var_pct_n = np.var(np.array(nnmodel.predict(X_test))) / np.var(Y_test)

print(train_err_n)
print(test_err_n)
print(train_var_pct_n)
print(test_var_pct_n)

0.04967249583777655
0.8459357037526326
0.9907820836544687
0.9232376763246279


### Tree

In [10]:
from sklearn import tree

# min_samples_split
Train_err_t = []
Test_err_t = []
for i in range(2,500,4): 
    tmodel = tree.DecisionTreeRegressor(min_samples_split=i,random_state=66) 
    tmodel.fit(X_train,Y_train)
    Train_err_t.append(np.sqrt(np.mean((tmodel.predict(X_train) - Y_train)**2)))
    Test_err_t.append(np.sqrt(np.mean((tmodel.predict(X_test) - Y_test)**2)))
    
import matplotlib.pyplot as plt

plt.plot(range(2,500,4),Train_err_t)
plt.title("Training Error of Decision Tree")
plt.xlabel("Min_Samples_Split")
plt.ylabel("Error")
plt.savefig("tree-train-err.png")
plt.close()

plt.plot(range(2,500,4),Test_err_t)
plt.title("Test Error of Decision Tree")
plt.xlabel("Min_Samples_Split")
plt.ylabel("Error")
plt.savefig("tree-test-err.png")
plt.close()

# Results
import graphviz 

# Select a best min_samples_split and random_state
tmodel = tree.DecisionTreeRegressor(min_samples_split=162,random_state=66)
tmodel.fit(X_train,Y_train)

train_err_t = np.sqrt(np.mean((tmodel.predict(X_train) - Y_train)**2))
test_err_t = np.sqrt(np.mean((tmodel.predict(X_test) - Y_test)**2))

dot_data = tree.export_graphviz(tmodel, out_file=None) 
graph = graphviz.Source(dot_data)
graph
graph.save("tree.gv")

train_var_pct_t = np.var(np.array(tmodel.predict(X_train))) / np.var(Y_train)
test_var_pct_t = np.var(np.array(tmodel.predict(X_test))) / np.var(Y_test)

### Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor as rf

Train_err_rf = []
Test_err_rf = []
Oob_err_rf = []

for i in range(20,1000,20): 
    fmodel = rf(n_estimators=i, oob_score=True, random_state=66, n_jobs=3)
    fmodel.fit(X_train,Y_train)
    Train_err_rf.append(np.sqrt(np.mean((fmodel.predict(X_train) - Y_train)**2)))
    Test_err_rf.append(np.sqrt(np.mean((fmodel.predict(X_test) - Y_test)**2)))
    Oob_err_rf.append(fmodel.oob_score_)

import matplotlib.pyplot as plt

plt.plot(range(20,1000,20),Train_err_rf)
plt.title("Training Error of Random Forest")
plt.xlabel("Number of trees")
plt.ylabel("Error")
plt.savefig("rf-train-err.png")
plt.close()

plt.plot(range(20,1000,20), np.array(Oob_err_rf))
plt.title("OOB Score of Random Forest")
plt.xlabel("Number of Trees")
plt.ylabel("Error")
plt.savefig("rf-oob-err.png")
plt.close()

/Users/yuchengjin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [20]:
plt.plot(range(20,1000,20),Test_err_rf)
plt.title("Test Error of Random Forest")
plt.xlabel("Number of trees")
plt.ylabel("Error")
plt.savefig("rf-test-err.png")
plt.close()

In [21]:
# Results
fmodel = rf(n_estimators=900, oob_score=True, random_state=66)
fmodel.fit(X_train,Y_train)
importance = fmodel.feature_importances_
oob_score_f = fmodel.oob_score_
train_err_f = np.sqrt(np.mean((fmodel.predict(X_train) - Y_train)**2))
test_err_f = np.sqrt(np.mean((fmodel.predict(X_test) - Y_test)**2))
train_var_pct_f = np.var(np.array(fmodel.predict(X_train))) / np.var(Y_train)
test_var_pct_f = np.var(np.array(fmodel.predict(X_test))) / np.var(Y_test)

print(oob_score_f)
print(train_err_f)
print(test_err_f)
print(train_var_pct_f)
print(test_var_pct_f)

0.22415209792433233
0.24742397597863375
0.6282805506274968
0.5450012057253169
0.2809824300209125


In [39]:
coef = []
coef.extend(list(importance))
cols = []
cols.extend(colnames)
coef = pd.DataFrame(coef, index = cols, columns = ["coefficient"])
coef.to_csv("rf_regression.csv")

### Bagging

In [12]:
from sklearn.ensemble import BaggingRegressor as bag

Train_err_b = []
Test_err_b = []
Oob_err_b = []
for i in range(20,500,10):  
    bmodel = bag(n_estimators=i, oob_score=True, random_state=66, n_jobs=3)
    bmodel.fit(X_train,Y_train)
    Train_err_b.append(np.sqrt(np.mean((bmodel.predict(X_train) - Y_train)**2)))
    Test_err_b.append(np.sqrt(np.mean((bmodel.predict(X_test) - Y_test)**2)))
    Oob_err_b.append(bmodel.oob_score_)

import matplotlib.pyplot as plt

plt.plot(range(20,500,10),Train_err_b)
plt.title("Train Error of Bagging")
plt.xlabel("Number of Trees")
plt.ylabel("Error")
plt.savefig("bagging-train-err.png")
plt.close()

plt.plot(range(20,500,10),Test_err_b)
plt.title("Test Error of Bagging")
plt.xlabel("Number of Trees")
plt.ylabel("Error")
plt.savefig("bagging-test-err.png")
plt.close()

plt.plot(range(20,500,10),-np.array(Oob_err_b))
plt.title("Oob Score of Bagging")
plt.xlabel("Number of Trees")
plt.ylabel("Error")
plt.savefig("bagging-oob-err.png")
plt.close()

/Users/yuchengjin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:1011: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
